# 08. Mini GPT 구현 - 전체 조립

## 학습 목표
- 지금까지 구현한 모든 컴포넌트 조립
- Weight Tying 기법 이해
- Autoregressive Text Generation 구현
- Decoding 전략 (Greedy, Top-k, Top-p) 이해

## 관련 면접 질문
- GPT-2의 파라미터 수는 어떻게 결정되나요?
- Weight Tying이란 무엇이고 왜 사용하나요?
- Temperature, Top-k, Top-p의 역할은?

---
## Pre-Quiz

### Q1. GPT 모델의 핵심 구성요소를 순서대로 나열하세요
```
입력 → ___ → ___ → ___ → ___ → 출력
```

### Q2. Weight Tying이란?
```
당신의 답: 
```

### Q3. Temperature가 높으면 생성 결과는 어떻게 변하나요?
```
당신의 답: 
```

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
from typing import Optional
import matplotlib.pyplot as plt

torch.manual_seed(42)

# matplotlib 한글 설정 (macOS)
plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
# 이전에 구현한 컴포넌트들 (제공)
def scaled_dot_product_attention(query, key, value, mask=None):
    d_k = query.size(-1)
    scores = torch.bmm(query, key.transpose(-2, -1)) / math.sqrt(d_k)
    if mask is not None:
        scores = scores.masked_fill(mask, float('-inf'))
    weights = F.softmax(scores, dim=-1)
    return torch.bmm(weights, value), weights

class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads, dropout=0.0):
        super().__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads
        
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, query, key, value, mask=None):
        batch_size = query.size(0)
        seq_q, seq_k = query.size(1), key.size(1)
        
        Q = self.W_q(query).view(batch_size, seq_q, self.num_heads, self.d_k).transpose(1, 2)
        K = self.W_k(key).view(batch_size, seq_k, self.num_heads, self.d_k).transpose(1, 2)
        V = self.W_v(value).view(batch_size, seq_k, self.num_heads, self.d_k).transpose(1, 2)
        
        Q = Q.contiguous().view(batch_size * self.num_heads, seq_q, self.d_k)
        K = K.contiguous().view(batch_size * self.num_heads, seq_k, self.d_k)
        V = V.contiguous().view(batch_size * self.num_heads, seq_k, self.d_k)
        
        if mask is not None and mask.dim() == 2:
            mask = mask.unsqueeze(0).expand(batch_size * self.num_heads, -1, -1)
        
        output, weights = scaled_dot_product_attention(Q, K, V, mask)
        
        output = output.view(batch_size, self.num_heads, seq_q, self.d_k)
        output = output.transpose(1, 2).contiguous().view(batch_size, seq_q, self.d_model)
        
        return self.dropout(self.W_o(output)), weights.view(batch_size, self.num_heads, seq_q, seq_k)

class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff=None, dropout=0.1):
        super().__init__()
        if d_ff is None:
            d_ff = d_model * 4
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        return self.dropout(self.fc2(self.dropout(F.gelu(self.fc1(x)))))

def create_causal_mask(seq_len, device=None):
    mask = torch.triu(torch.ones(seq_len, seq_len), diagonal=1).bool()
    if device is not None:
        mask = mask.to(device)
    return mask

class DecoderBlock(nn.Module):
    def __init__(self, d_model, num_heads, d_ff=None, dropout=0.1):
        super().__init__()
        self.attention = MultiHeadAttention(d_model, num_heads, dropout)
        self.ffn = PositionWiseFeedForward(d_model, d_ff, dropout)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x, mask=None):
        seq_len = x.size(1)
        causal_mask = create_causal_mask(seq_len, x.device)
        if mask is not None:
            combined_mask = causal_mask | mask
        else:
            combined_mask = causal_mask
        
        normed = self.norm1(x)
        attn_out, _ = self.attention(normed, normed, normed, combined_mask)
        x = x + self.dropout(attn_out)
        
        normed = self.norm2(x)
        ffn_out = self.ffn(normed)
        x = x + ffn_out
        
        return x

---
## 1. GPT 설정 클래스

In [ ]:
class MiniGPTConfig:
    """GPT 설정"""
    
    def __init__(
        self,
        vocab_size: int = 50257,  # GPT-2 vocab size
        d_model: int = 768,
        num_heads: int = 12,
        num_layers: int = 12,
        d_ff: int = None,  # default: d_model * 4
        max_len: int = 1024,
        dropout: float = 0.1,
    ):
        self.vocab_size = vocab_size
        self.d_model = d_model
        self.num_heads = num_heads
        self.num_layers = num_layers
        self.d_ff = d_ff if d_ff is not None else d_model * 4
        self.max_len = max_len
        self.dropout = dropout

# GPT-2 크기별 설정
GPT2_CONFIGS = {
    'small': MiniGPTConfig(d_model=768, num_heads=12, num_layers=12),    # 117M
    'medium': MiniGPTConfig(d_model=1024, num_heads=16, num_layers=24),  # 345M
    'large': MiniGPTConfig(d_model=1280, num_heads=20, num_layers=36),   # 762M
    'xl': MiniGPTConfig(d_model=1600, num_heads=25, num_layers=48),      # 1.5B
}

print("GPT-2 크기별 설정:")
for name, config in GPT2_CONFIGS.items():
    print(f"  {name}: d_model={config.d_model}, heads={config.num_heads}, layers={config.num_layers}")

---
## 2. Mini GPT 구현

### 전체 구조
```
token_ids → Token Embedding → + → Dropout → DecoderBlock × N → LayerNorm → LM Head → logits
                              ↑
            Position Embedding
```

### Weight Tying
Token Embedding과 LM Head의 weight를 공유하는 기법:
- 파라미터 수 감소 (vocab_size × d_model 절약)
- 성능 향상 효과도 있음
- GPT-2, GPT-3 등에서 사용

In [ ]:
# =================================================
# TODO: Mini GPT 구현
# =================================================
#
# 구현 순서:
# 1. __init__: 모든 레이어 초기화
# 2. forward: 순전파 (loss 계산 포함)
# 3. generate: autoregressive 텍스트 생성
#
# =================================================

class MiniGPT(nn.Module):
    def __init__(self, config: MiniGPTConfig):
        super().__init__()
        self.config = config
        
        # ============================================
        # 여기에 구현하세요
        # - token_embedding: nn.Embedding(vocab_size, d_model)
        # - position_embedding: nn.Embedding(max_len, d_model)
        # - dropout: nn.Dropout
        # - blocks: nn.ModuleList of DecoderBlock
        # - ln_f: nn.LayerNorm (final)
        # - lm_head: nn.Linear(d_model, vocab_size, bias=False)
        # 
        # (선택) Weight Tying:
        # self.lm_head.weight = self.token_embedding.weight
        # ============================================
        
        pass  # 이 줄을 지우고 구현
    
    def forward(
        self,
        input_ids: torch.Tensor,
        targets: Optional[torch.Tensor] = None
    ) -> tuple[torch.Tensor, Optional[torch.Tensor]]:
        """
        Args:
            input_ids: (batch, seq) - 입력 토큰 ID
            targets: (batch, seq) - 타겟 토큰 ID (학습 시)
            
        Returns:
            logits: (batch, seq, vocab_size)
            loss: cross entropy loss (targets가 있는 경우)
        """
        # ============================================
        # 여기에 구현하세요
        # 1. token embedding
        # 2. position ids 생성 및 embedding
        # 3. dropout(tok_emb + pos_emb)
        # 4. decoder blocks 순차 통과
        # 5. final layer norm
        # 6. lm_head로 logits 생성
        # 7. loss 계산 (targets 있는 경우)
        #    - shift: logits[..., :-1, :], targets[..., 1:]
        #    - cross_entropy
        # ============================================
        
        pass  # 이 줄을 지우고 구현

In [ ]:
# ============ 테스트 ============
def test_mini_gpt_forward():
    config = MiniGPTConfig(
        vocab_size=1000,
        d_model=64,
        num_heads=4,
        num_layers=2,
        max_len=128
    )
    
    model = MiniGPT(config)
    
    batch, seq = 2, 10
    input_ids = torch.randint(0, config.vocab_size, (batch, seq))
    targets = torch.randint(0, config.vocab_size, (batch, seq))
    
    # Forward without targets
    logits, loss = model(input_ids)
    assert logits.shape == (batch, seq, config.vocab_size)
    assert loss is None
    
    # Forward with targets
    logits, loss = model(input_ids, targets)
    assert logits.shape == (batch, seq, config.vocab_size)
    assert loss is not None
    
    print("✅ MiniGPT Forward 테스트 통과!")
    print(f"   Logits shape: {logits.shape}")
    print(f"   Loss: {loss.item():.4f}")
    
    # 파라미터 수 계산
    num_params = sum(p.numel() for p in model.parameters())
    print(f"   Parameters: {num_params:,}")

test_mini_gpt_forward()

---
## 3. Text Generation 구현

### Decoding 전략들

| 전략 | 설명 | 특징 |
|------|------|------|
| **Greedy** | argmax 선택 | 결정적, 반복 문제 |
| **Temperature** | logits/T 후 sampling | T↑ 다양성↑ |
| **Top-k** | 상위 k개만 sampling | 확률 낮은 토큰 제거 |
| **Top-p (Nucleus)** | 누적확률 p까지 sampling | 동적 후보 수 |

In [ ]:
# =================================================
# TODO: generate 메서드를 MiniGPT에 추가
# =================================================
#
# 아래 코드를 MiniGPT 클래스에 메서드로 추가하세요
# (또는 함수로 구현해도 됩니다)
#
# =================================================

@torch.no_grad()
def generate(
    model: MiniGPT,
    input_ids: torch.Tensor,
    max_new_tokens: int = 50,
    temperature: float = 1.0,
    top_k: int = None,
    top_p: float = None,
) -> torch.Tensor:
    """
    Autoregressive 텍스트 생성
    
    Args:
        model: MiniGPT 모델
        input_ids: (batch, seq) - 시작 토큰들
        max_new_tokens: 생성할 최대 토큰 수
        temperature: sampling temperature (0에 가까울수록 greedy)
        top_k: top-k sampling (None이면 사용 안 함)
        top_p: nucleus sampling (None이면 사용 안 함)
        
    Returns:
        generated: (batch, seq + max_new_tokens)
    """
    model.eval()
    
    # ============================================
    # 여기에 구현하세요
    # 
    # for _ in range(max_new_tokens):
    #     1. 현재 input_ids로 forward (마지막 위치 logits만 필요)
    #     2. temperature 적용: logits = logits / temperature
    #     3. top_k filtering (있는 경우)
    #        - 상위 k개 외 -inf로 설정
    #     4. top_p filtering (있는 경우)
    #        - 누적 확률 p 이후 -inf로 설정
    #     5. softmax → 확률 분포
    #     6. multinomial sampling (또는 greedy)
    #     7. 새 토큰을 input_ids에 concat
    # ============================================
    
    pass  # 이 줄을 지우고 구현

In [ ]:
# Top-k, Top-p 필터링 헬퍼 함수 (힌트)
def top_k_filtering(logits, top_k):
    """Top-k 필터링: 상위 k개 외 -inf로 설정"""
    if top_k is None or top_k <= 0:
        return logits
    
    values, _ = torch.topk(logits, top_k)
    min_value = values[:, -1].unsqueeze(-1)
    return torch.where(logits < min_value, torch.tensor(float('-inf')), logits)


def top_p_filtering(logits, top_p):
    """Top-p (Nucleus) 필터링: 누적 확률 p 이후 -inf로 설정"""
    if top_p is None or top_p >= 1.0:
        return logits
    
    sorted_logits, sorted_indices = torch.sort(logits, descending=True)
    cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)
    
    # 누적 확률이 p를 넘는 토큰들 마스킹
    sorted_indices_to_remove = cumulative_probs > top_p
    # 첫 번째 토큰은 항상 유지
    sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
    sorted_indices_to_remove[..., 0] = False
    
    indices_to_remove = sorted_indices_to_remove.scatter(
        dim=-1, index=sorted_indices, src=sorted_indices_to_remove
    )
    logits[indices_to_remove] = float('-inf')
    return logits

In [ ]:
# ============ 테스트 ============
def test_generate():
    config = MiniGPTConfig(
        vocab_size=100,
        d_model=64,
        num_heads=4,
        num_layers=2,
        max_len=128
    )
    
    model = MiniGPT(config)
    
    # 시작 토큰
    prompt = torch.randint(0, config.vocab_size, (1, 5))
    
    # 다양한 decoding 전략 테스트
    print("Generation 테스트:")
    print(f"  Prompt shape: {prompt.shape}")
    
    # Greedy
    greedy_output = generate(model, prompt, max_new_tokens=10, temperature=0.001)
    print(f"  Greedy output shape: {greedy_output.shape}")
    
    # Temperature sampling
    temp_output = generate(model, prompt, max_new_tokens=10, temperature=0.8)
    print(f"  Temperature(0.8) output shape: {temp_output.shape}")
    
    # Top-k
    topk_output = generate(model, prompt, max_new_tokens=10, temperature=1.0, top_k=10)
    print(f"  Top-k(10) output shape: {topk_output.shape}")
    
    # Top-p
    topp_output = generate(model, prompt, max_new_tokens=10, temperature=1.0, top_p=0.9)
    print(f"  Top-p(0.9) output shape: {topp_output.shape}")
    
    print("\n✅ Generate 테스트 통과!")

test_generate()

---
## 4. 파라미터 수 분석

In [ ]:
def count_parameters(model, verbose=False):
    """모델 파라미터 수 분석"""
    total = 0
    breakdown = {}
    
    for name, param in model.named_parameters():
        count = param.numel()
        total += count
        
        # 컴포넌트별 그룹핑
        component = name.split('.')[0]
        breakdown[component] = breakdown.get(component, 0) + count
    
    if verbose:
        print(f"\n파라미터 분석:")
        for comp, count in sorted(breakdown.items(), key=lambda x: -x[1]):
            pct = count / total * 100
            print(f"  {comp}: {count:,} ({pct:.1f}%)")
        print(f"  ---------------")
        print(f"  Total: {total:,}")
    
    return total, breakdown

# GPT-2 크기별 파라미터 수 비교
print("GPT-2 크기별 파라미터 수:")
for name, config in GPT2_CONFIGS.items():
    model = MiniGPT(config)
    total, _ = count_parameters(model)
    print(f"  {name}: {total/1e6:.1f}M parameters")

In [ ]:
# 상세 분석 (small 모델)
config = MiniGPTConfig(
    vocab_size=50257,
    d_model=768,
    num_heads=12,
    num_layers=12
)
model = MiniGPT(config)
count_parameters(model, verbose=True)

---
## 5. Decoding 전략 비교 시각화

In [ ]:
def visualize_decoding_strategies():
    """Decoding 전략별 확률 분포 시각화"""
    torch.manual_seed(42)
    
    # 가상의 logits (vocab_size=20)
    logits = torch.randn(1, 20) * 2
    vocab = [f'tok_{i}' for i in range(20)]
    
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    
    # 1. Original (temperature=1)
    probs_orig = F.softmax(logits, dim=-1).squeeze()
    axes[0, 0].bar(range(20), probs_orig.numpy())
    axes[0, 0].set_title('Original (T=1.0)')
    axes[0, 0].set_xlabel('Token')
    axes[0, 0].set_ylabel('Probability')
    
    # 2. Low Temperature (greedy-like)
    probs_low = F.softmax(logits / 0.3, dim=-1).squeeze()
    axes[0, 1].bar(range(20), probs_low.numpy())
    axes[0, 1].set_title('Low Temperature (T=0.3)')
    axes[0, 1].set_xlabel('Token')
    axes[0, 1].set_ylabel('Probability')
    
    # 3. High Temperature (diverse)
    probs_high = F.softmax(logits / 2.0, dim=-1).squeeze()
    axes[1, 0].bar(range(20), probs_high.numpy())
    axes[1, 0].set_title('High Temperature (T=2.0)')
    axes[1, 0].set_xlabel('Token')
    axes[1, 0].set_ylabel('Probability')
    
    # 4. Top-k (k=5)
    logits_topk = top_k_filtering(logits.clone(), top_k=5)
    probs_topk = F.softmax(logits_topk, dim=-1).squeeze()
    axes[1, 1].bar(range(20), probs_topk.numpy())
    axes[1, 1].set_title('Top-k (k=5)')
    axes[1, 1].set_xlabel('Token')
    axes[1, 1].set_ylabel('Probability')
    
    plt.tight_layout()
    plt.show()

visualize_decoding_strategies()

---
## Post-Quiz

### Q1. Weight Tying을 사용하면 파라미터가 얼마나 줄어드나요? (vocab_size=50257, d_model=768)
```
당신의 답: 
```

### Q2. Temperature=0.5 vs Temperature=1.5의 생성 결과 차이는?
```
당신의 답: 
```

### Q3. Top-k=50, Top-p=0.9를 함께 사용하면 어떻게 동작하나요?
```
당신의 답: 
```

---
## 정답

<details>
<summary>👉 클릭하여 정답 코드 보기</summary>

```python
class MiniGPT(nn.Module):
    def __init__(self, config: MiniGPTConfig):
        super().__init__()
        self.config = config
        
        self.token_embedding = nn.Embedding(config.vocab_size, config.d_model)
        self.position_embedding = nn.Embedding(config.max_len, config.d_model)
        self.dropout = nn.Dropout(config.dropout)
        
        self.blocks = nn.ModuleList([
            DecoderBlock(config.d_model, config.num_heads, config.d_ff, config.dropout)
            for _ in range(config.num_layers)
        ])
        
        self.ln_f = nn.LayerNorm(config.d_model)
        self.lm_head = nn.Linear(config.d_model, config.vocab_size, bias=False)
        
        # Weight Tying (선택)
        self.lm_head.weight = self.token_embedding.weight
    
    def forward(self, input_ids, targets=None):
        batch, seq_len = input_ids.shape
        device = input_ids.device
        
        # Embeddings
        tok_emb = self.token_embedding(input_ids)
        pos_ids = torch.arange(seq_len, device=device).unsqueeze(0)
        pos_emb = self.position_embedding(pos_ids)
        
        x = self.dropout(tok_emb + pos_emb)
        
        # Decoder blocks
        for block in self.blocks:
            x = block(x)
        
        # Output
        x = self.ln_f(x)
        logits = self.lm_head(x)
        
        # Loss
        loss = None
        if targets is not None:
            # Shift: logits[:-1] 예측 → targets[1:]
            shift_logits = logits[..., :-1, :].contiguous()
            shift_targets = targets[..., 1:].contiguous()
            loss = F.cross_entropy(
                shift_logits.view(-1, self.config.vocab_size),
                shift_targets.view(-1)
            )
        
        return logits, loss


@torch.no_grad()
def generate(model, input_ids, max_new_tokens=50, temperature=1.0, top_k=None, top_p=None):
    model.eval()
    
    for _ in range(max_new_tokens):
        # max_len 제한
        idx_cond = input_ids[:, -model.config.max_len:]
        
        # Forward
        logits, _ = model(idx_cond)
        logits = logits[:, -1, :]  # 마지막 위치만
        
        # Temperature
        logits = logits / max(temperature, 1e-8)
        
        # Top-k filtering
        if top_k is not None:
            logits = top_k_filtering(logits, top_k)
        
        # Top-p filtering
        if top_p is not None:
            logits = top_p_filtering(logits, top_p)
        
        # Sampling
        probs = F.softmax(logits, dim=-1)
        next_token = torch.multinomial(probs, num_samples=1)
        
        # Concat
        input_ids = torch.cat([input_ids, next_token], dim=-1)
    
    return input_ids
```

</details>

<details>
<summary>👉 Pre-Quiz 정답</summary>

### Q1. GPT 구성요소
입력 → **Token Embedding + Position Embedding** → **Dropout** → **DecoderBlock × N** → **LayerNorm** → **LM Head** → 출력

### Q2. Weight Tying이란?
Token Embedding과 LM Head(출력 projection)의 가중치를 **공유**하는 기법. 파라미터 수 절약 + 입력/출력 임베딩 공간 일관성 유지.

### Q3. Temperature 높으면?
logits를 T로 나누면 softmax 분포가 **더 균등**해짐 → 다양한 토큰이 선택될 확률↑ → 창의적이지만 일관성↓

</details>

<details>
<summary>👉 Post-Quiz 정답</summary>

### Q1. Weight Tying 파라미터 절약
vocab_size × d_model = 50257 × 768 = **38,597,376** ≈ **38.6M** 파라미터 절약

### Q2. Temperature 차이
- **T=0.5**: 확률 분포가 더 뾰족해짐 → 높은 확률 토큰 위주 → **결정적, 반복 경향**
- **T=1.5**: 확률 분포가 더 평평해짐 → 다양한 토큰 선택 → **다양하지만 일관성↓**

### Q3. Top-k + Top-p 함께 사용
두 필터링이 **순차적**으로 적용됨. 먼저 Top-k로 후보를 k개로 제한 → 그 중 Top-p로 누적확률 p까지만 유지. 더 **엄격한 필터링** 효과.

</details>